Author: Matteo Borzi 280104 `<matteo.borzi@studenti.polito.it>`  
Repository: [https://github.com/matteoborzi/computational-intelligence](https://github.com/matteoborzi/computational-intelligence)

Original code:
>Copyright **`(c)`** 2021 Giovanni Squillero `<squillero@polito.it>`  
>[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
>Free for personal or classroom use; see 'LICENCE.md' for details.

# Connect 4

In [13]:
from collections import Counter
import numpy as np

In [14]:
NUM_COLUMNS = 7
COLUMN_HEIGHT = 6
FOUR = 4
GAME_STATUS = False
HUMAN = 1
AI = -1
DEPTH = 6
PLAYERS = {1: "Human", -1:"Computer"}
GAME_RESULT = {1: "Human wins!", -1: "Computer wins!", 0: "No moves left"}

# Board can be initiatilized with `board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)`
# Notez Bien: Connect 4 "columns" are actually NumPy "rows"

## Basic Functions

In [15]:
def valid_moves(board):
    """Returns columns where a disc may be played"""
    return [n for n in range(NUM_COLUMNS) if board[n, COLUMN_HEIGHT - 1] == 0]


def play(board, column, player):
    """Updates `board` as `player` drops a disc in `column`"""
    (index, ) = next((i for i, v in np.ndenumerate(board[column]) if v == 0))
    board[column, index] = player


def take_back(board, column):
    """Updates `board` removing top disc from `column`"""
    (index, ) = [i for i, v in np.ndenumerate(board[column]) if v != 0][-1]
    board[column, index] = 0


def board_full(board):
    return np.count_nonzero(board == 0) == 0


def count_horizontal_seq(board, player, length=4):
    return sum([
        all(board[c, r] == player) for c in range(NUM_COLUMNS)
        for r in (list(range(n, n + length))
                  for n in range(COLUMN_HEIGHT - length + 1))
    ])


def count_vertical_seq(board, player, length=4):
    return sum([
        all(board[c, r] == player) for r in range(COLUMN_HEIGHT)
        for c in (list(range(n, n + length))
                  for n in range(NUM_COLUMNS - length + 1))
    ])


def count_diag_seq(board, player, length=4):
    return sum([
        np.all(board[diag] == player)
        for diag in ((range(ro, ro + length), range(co, co + length))
                     for ro in range(0, NUM_COLUMNS - length + 1)
                     for co in range(0, COLUMN_HEIGHT - length + 1))
    ])


def count_neg_diag_seq(board, player, length=4):
    return sum([
        np.all(board[diag] == player)
        for diag in ((range(ro, ro + length), range(co + length - 1, co - 1, -1))
                     for ro in range(0, NUM_COLUMNS - length + 1)
                     for co in range(0, COLUMN_HEIGHT - length + 1))
    ])


def four_in_a_row(board, player):
    """Checks if `player` has a 4-piece line"""
    return (any(
        all(board[c, r] == player) for c in range(NUM_COLUMNS)
        for r in (list(range(n, n + FOUR))
                  for n in range(COLUMN_HEIGHT - FOUR + 1))) or any(
                      all(board[c, r] == player) for r in range(COLUMN_HEIGHT)
                      for c in (list(range(n, n + FOUR))
                                for n in range(NUM_COLUMNS - FOUR + 1)))
            or any(
                np.all(board[diag] == player)
                for diag in ((range(ro, ro + FOUR), range(co, co + FOUR))
                             for ro in range(0, NUM_COLUMNS - FOUR + 1)
                             for co in range(0, COLUMN_HEIGHT - FOUR + 1)))
            or any(
                np.all(board[diag] == player)
                for diag in ((range(ro, ro + FOUR),
                              range(co + FOUR - 1, co - 1, -1))
                             for ro in range(0, NUM_COLUMNS - FOUR + 1)
                             for co in range(0, COLUMN_HEIGHT - FOUR + 1))))


## Minimax with alpha-beta pruning

In [16]:
def minmax(board):
    valid = valid_moves(board)
    np.random.shuffle(valid)
    best  = valid[0]
    best_score = float("-inf")

    player = AI
    opponent = HUMAN

    alpha = float("-inf")
    beta = float("inf")
  
    for move in valid:
        new_board = board.copy()
        play(new_board, move, player)
        board_score = min_beta(new_board, DEPTH, player, opponent, alpha, beta)
        if board_score > best_score:
            best_score = board_score
            best = move
    return best


def eval_board(board):
    if four_in_a_row(board, HUMAN):
        # Player won
        return 1
    elif four_in_a_row(board, AI):
        # Computer won
        return -1
    else:
        # Not terminal
        return 0

### Minimize function (beta)

In [ ]:
def min_beta(board, depth, player, opponent, a, b):
    valid = []
    for col in range(NUM_COLUMNS):
        if col in valid_moves(board):
            play(board, col, player)
            valid.append(col)

    if depth == 0 or len(valid) == 0 or four_in_a_row(board, player) or four_in_a_row(board, opponent):
        return get_score(board, player)
    
    valid = valid_moves(board) 
    beta = b
    
    for move in valid:
        board_score = float("inf")
        if a < beta:
            new_board = board.copy()
            play(new_board, move, player)
            board_score = max_alpha(new_board, depth - 1, player, opponent, a, beta)

        if board_score < beta:
            beta = board_score
    return beta

### Maximize function (alpha)

In [ ]:
def max_alpha(board, depth, player, opponent, a, b):
    valid = []
    for col in range(NUM_COLUMNS):
        if col in valid_moves(board):
            play(board, col, player)
            valid.append(col)

    if depth == 0 or len(valid) == 0 or four_in_a_row(board, opponent) or four_in_a_row(board, player):
        return get_score(board, player)

    alpha = a        
    for move in valid:
        board_score = float("-inf")
        if alpha < b:
            new_board = board.copy()
            play(new_board, move, player)
            board_score = min_beta(new_board, depth - 1, player, opponent, alpha, b)

        if board_score > alpha:
            alpha = board_score
    return alpha

### Score function

In [ ]:
def count(board, player, length):
    return count_horizontal_seq(board, player, length) + count_vertical_seq(
        board, player, length) + count_diag_seq(
            board, player, length) + count_neg_diag_seq(board, player, length)


def get_score(board, player):
    opponent = player * -1
    player_score = count(board, player, 4) * 99999 + count(board, player, 3) * 999 + count(board, player, 2) * 99
    opponent_score =  count(board, opponent, 4) * 99999 + count(board, opponent, 3) * 999 + count(board, opponent, 2) * 99

    if four_in_a_row(board, opponent):
        return float('-inf')
    else:
        return player_score - opponent_score

## Turn functions

In [17]:
def player_turn(board):
    move = None
    while move is None or move not in valid_moves(board):
        try:
            move = int(input(f"\nPlayer turn. Choose a column (0-{NUM_COLUMNS-1}): "))
        except ValueError:
            print("Please insert a integer value")
            continue
        if move not in valid_moves(board):
            print("Column not valid! Pleas try again")
    play(board, move, HUMAN)
    print(board)       
    return board
    
def AI_turn(board):
    print("\nComputer turn.")
    move = minmax(board)
    play(board, move, AI)
    print(board)
    return board

## Main function

In [18]:
board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)
#turns = np.random.choice([1,-1])
turns = HUMAN
GAME_STATUS = True
print(f"Game starting! First player is {PLAYERS[turns]}")
while True:
    if board_full(board):
        print("No moves left")
        break

    if turns == HUMAN:
        board = player_turn(board)
    else:
        board = AI_turn(board)

    turn_result = eval_board(board)

    if turn_result == 1:
        print("Human wins!")
        break
    elif turn_result == -1:
        print("Computer wins!")
        break
    
    turns *= -1

Game starting! First player is Human
